### Read CSV

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
csv_headers = ["center", "left", "right", "steering", "throttle", "brake", "speed"]

data = pd.read_csv('CarSim_data/driving_log.csv', names=csv_headers)

In [ ]:
data.head()

In [ ]:
data['center'][2]

In [ ]:
A.shape

### Read Image, Crop and Resize

In [ ]:
filename = data['center'][1]
#print(file)

I = cv2.imread(filename)
b,g,r = cv2.split(I)
I = cv2.merge([r,g,b])
#cv2.imwrite('I.jpg',I)
Im = I[50:,:]
Im = cv2.resize(Im,None,fx=0.25,fy=0.25,interpolation=cv2.INTER_AREA)
plt.imshow(Im)

In [ ]:
Im.shape

### Create Features and Labels

### Preprocess the Data
1.) Shuffle the data

2.) Normalize the features using Min-Max scaling between -0.5 and 0.5

3.) Steering Angle: -25 to 25


In [ ]:
data.tail()

In [ ]:
data.shape

In [ ]:
# Import Training Data from CSV File
import csv

In [ ]:
# Main Driving Log
data_c = csv.reader(open('CarSim_data/driving_log.csv'), delimiter=",",quotechar='|')


# Create Empty Arrays for Center Driving Log Data
img_center = []
img_left = []
img_right = []
steering = []

In [ ]:
# Populate Center Driving Arrays
for row in data_c:
      img_center.append(row[0])
      img_left.append(row[1])
      img_right.append(row[2])
      steering.append(row[3])

In [ ]:
print(img_center)

In [ ]:
# Center Array
img_center = np.asarray(img_center)
img_left = np.asarray(img_left)
img_right = np.asarray(img_right)

steering = np.asarray(steering, dtype=np.float32)

In [ ]:
img_center.shape

In [ ]:

arr = np.zeros((num_examples, width, height, channels))
arr[i, :, :, :] = img
